In [ ]:
import scipy.io
import scipy
import numpy as np
from sklearn.model_selection import train_test_split
import torch,torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader,TensorDataset
from sklearn.preprocessing import StandardScaler

In [ ]:
## Fixed params
eff=0.5
nu=1
PL_d0_dB = 31.67
alpha = 2
Z_stdDev = 2
W=1e6
N0=1e-14
D=50

scale_factor=1e7
numChReal=1000000
P_AP=2
p_max=0.001

In [ ]:
input_data_folder='../input_data'
output_data_folder='../output/pure_DL_models/'
tensor_board_folder='../runs/pureDL'

In [ ]:
def power_to_time_tensor(p_i,gh,gt,D,W,eff,P_AP,N0):
    t_i=D/(W*torch.log2(1+(p_i*gt)/(N0*W)))
    req_harv_energy= t_i*p_i
    t0=torch.max(req_harv_energy/(eff*P_AP*gh),dim=1)[0]
    #tot_time=torch.sum(t_i,dim=1)+t0
    return scale_factor*torch.cat((t0.unsqueeze(dim=1),t_i),dim=1)

In [ ]:
match = scipy.io.loadmat(input_data_folder+'/channel_gains.mat')
test_ind=np.load(input_data_folder+'/test_ind.npy')

In [ ]:
p_max

In [ ]:
dnn_result=[]
for num_of_user in range(2,5,2):
    ## DNN Arch  ##
    class TimeNet(nn.Module):

        def __init__(self):
            super(TimeNet, self).__init__()

            self.layer1 = nn.Linear(2*num_of_user,8*num_of_user)
            self.layer2 = nn.Linear(8*num_of_user,8*num_of_user)
            self.layer3 = nn.Linear(8*num_of_user,8*num_of_user)
            self.layer4 = nn.Linear(8*num_of_user,4*num_of_user)
            self.layer5 = nn.Linear(4*num_of_user,4*num_of_user)
            self.output = nn.Linear(4*num_of_user,num_of_user+1)

        def forward(self,input_x):

            x1_1 = F.relu(self.layer1(input_x))
            x = F.relu(self.layer2(x1_1))
            x = F.relu(self.layer3(x))
            x = F.relu(self.layer4(x))
            x = F.relu(self.layer5(x))
            x=torch.abs(self.output(x))

            return x
    class PowerNet(nn.Module):

        def __init__(self):
            super(PowerNet, self).__init__()

            self.layer1 = nn.Linear(2*num_of_user,8*num_of_user)
            self.layer2 = nn.Linear(8*num_of_user,8*num_of_user)
            self.layer3 = nn.Linear(8*num_of_user,8*num_of_user)
            self.layer4 = nn.Linear(8*num_of_user,4*num_of_user)
            self.layer5 = nn.Linear(4*num_of_user,4*num_of_user)
            self.output = nn.Linear(4*num_of_user,num_of_user)

        def forward(self,input_x):

            x1_1 = F.relu(self.layer1(input_x))
            x = F.relu(self.layer2(x1_1))
            x = F.relu(self.layer3(x))
            x = F.relu(self.layer4(x))
            x = F.relu(self.layer5(x))
            x=p_max*(scale_factor/100)*F.sigmoid(self.output(x))

            return x

    # read labels
    mat = scipy.io.loadmat(input_data_folder+'/result_compare_N014'+'_PAP'+str(P_AP)+'_M'+str(num_of_user)+'_pmax'+str(p_max)+'.mat')
    outs=mat['tot_time_MRTTMA4'].T
    outs=outs*scale_factor

    mat_pow=scipy.io.loadmat(input_data_folder+'/powers_N014'+'_PAP'+str(P_AP)+'_pmax'+str(p_max)+'_M'+str(num_of_user)+'.mat')
    outs_pow=mat_pow['p_i'].T
    outs_pow=outs_pow*scale_factor/100

    gh=match['gh_arr'][0:num_of_user,0:numChReal]
    gt=match['gt_arr'][0:num_of_user,0:numChReal]

    ins=np.concatenate((gh,gt)).T
    indicies=np.arange(len(ins))
    remaning_ind=list(set(indicies)-set(test_ind))

    ins2=ins[remaning_ind]
    outs2=outs[remaning_ind]
    outs_pow2=outs_pow[remaning_ind]

    X_test=ins[test_ind]
    y_test=outs[test_ind]
    y_pow_test=outs_pow[test_ind]
    idx_test=list(test_ind)

    X_train, X_val, y_train, y_val, y_pow_train, y_pow_val,idx_train,idx_val = train_test_split( ins2,outs2,outs_pow2, remaning_ind, test_size=0.01, random_state=42)

    # Scale
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_val = ss.transform(X_val)
    X_test = ss.transform(X_test)

    #Dataset & DataLoder
    torch_dataset = TensorDataset(torch.tensor(X_train.astype(np.float32)), torch.tensor(y_train.astype(np.float32)),torch.tensor(y_pow_train.astype(np.float32)))
    val_torch_dataset = TensorDataset(torch.tensor(X_val.astype(np.float32)), torch.tensor(y_val.astype(np.float32)), torch.tensor(y_pow_val.astype(np.float32)))



    train_data_loader = DataLoader(torch_dataset, batch_size=32)
    val_data_loader = DataLoader(val_torch_dataset,batch_size=1)


    ## TRAINING
    # default `log_dir` is "runs" - we'll be more specific here
    writer = SummaryWriter(tensor_board_folder+'/onlyDNN_abs_time_N014'+'_PAP'+str(P_AP)+'_M'+str(num_of_user)+'_pmax'+str(p_max))
    writer_pow = SummaryWriter(tensor_board_folder+'/onlyDNN_abs_pow_N014'+'_PAP'+str(P_AP)+'_M'+str(num_of_user)+'_pmax'+str(p_max))
    NO_EPOCHS = 20

    val_loss_best=10000000
    val_loss_best_pow=10000000

    model_time = TimeNet()
    model_pow = PowerNet()

    optimizer = Adam(model_time.parameters(),lr=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, 'min')

    optimizer_pow = Adam(model_pow.parameters(),lr=1e-4)
    scheduler_pow = ReduceLROnPlateau(optimizer_pow, 'min')

    loss_fn = nn.MSELoss()
    loss_fn_pow = nn.MSELoss()
   
    for epoch_idx in range(NO_EPOCHS):
            model_time.train()
            model_pow.train()
            epoch_loss = 0
            len_batches = 0
            epoch_loss_pow = 0

            for ii,sample in enumerate(tqdm(train_data_loader)):

                local_inp, local_tgt,local_tgt_pow = sample
                output = model_time(local_inp)
                output_pow = model_pow(local_inp)
                loss = loss_fn(output,local_tgt)
                loss_pow = loss_fn_pow(output_pow,local_tgt_pow)



                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                optimizer_pow.zero_grad()
                loss_pow.backward()
                optimizer_pow.step()

                #print(f'Train Epoch: {epoch_idx}, Batch: {ii*len(local_tgt)},Loss: {loss}')
                epoch_loss += loss.detach().numpy()
                epoch_loss_pow += loss_pow.detach().numpy()

            # Validation phase
            model_time.eval()
            val_loss = 0
            model_pow.eval()
            val_loss_pow = 0


            with torch.no_grad():
                for jj, val_sample in enumerate(val_data_loader):
                    local_inp, local_tgt, local_tgt_pow= val_sample
                    output = model_time(local_inp)
                    val_loss += loss_fn(output,local_tgt)

                    output_pow = model_pow(local_inp)
                    val_loss_pow += loss_fn(output_pow,local_tgt_pow)

                val_loss /= (jj+1)
                val_loss_pow /= (jj+1)


            if val_loss < val_loss_best:
                    val_loss_best=val_loss
                    torch.save(model_time, output_data_folder+'/model_sep_N'+str(num_of_user)+'.pt')
                    #X_test_inverse = torch.Tensor(ss.inverse_transform(X_test))
                    test_output = model_time((torch.tensor(X_test.astype(np.float32))))
                    test_output_pow = model_pow((torch.tensor(X_test.astype(np.float32))))


            writer.add_scalar('val_loss',
                        val_loss,
                        epoch_idx)

            writer.add_scalar('training loss',
                        epoch_loss / (ii+1),
                        epoch_idx)
            writer_pow.add_scalar('val_loss',
                        val_loss_pow,
                        epoch_idx)

            writer_pow.add_scalar('training loss',
                        epoch_loss_pow / (ii+1),
                        epoch_idx)

    np.save(output_data_folder+'/tot_time_DNN_N014'+'_PAP'+str(P_AP)+'_M'+str(num_of_user)+'_pmax'+str(p_max),(test_output/scale_factor).detach().numpy())
    np.save(output_data_folder+'/pow_DNN_N014'+'_PAP'+str(P_AP)+'_M'+str(num_of_user)+'_pmax'+str(p_max),(test_output_pow/scale_factor*100).detach().numpy())
    dnn_result.append((test_output/scale_factor).sum(axis=1).mean().detach().item())
#np.save(output_data_folder+'/all_tot_times_N014'+'_PAP'+str(P_AP)+'_M'+str(num_of_user)+'_pmax'+str(p_max),[powmu_result,maxeh_result,chi_result,dnn_result])




In [ ]:
import torch
def outage_calc(t_0,t_i,p_i,gh,gt,D,W,eff,P_AP,N0):
    energy_harvested=t_0*eff*P_AP*gh

    time_to_comsume_energy=energy_harvested/p_i

    if t_i>=time_to_comsume_energy:
        data_sent=time_to_comsume_energy*W*torch.log2(1+(p_i*gt)/(N0*W))
    else:
        data_sent=t_i*W*torch.log2(1+(p_i*gt)/(N0*W))
    #print('Data Sent' + str(data_sent))
    outage=D-data_sent

    return outage